In [110]:
from typing import Optional
from pydantic import Field
from typing_extensions import TypedDict


class ExpenseSchema(TypedDict):
    date: str
    amount: int
    category: str
    description: Optional[str]


tools = [ExpenseSchema]

In [111]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")
llm_tools = llm.bind_tools(tools)

In [112]:
from datetime import datetime

user_input = "I spent 1000 taka on groceries"

res = llm_tools.invoke(user_input).tool_calls[0]["args"]
res["date"] = datetime.now().strftime("%Y-%m-%d")

# Replace null values with "Null"
if not res.get("category"):
    res["category"] = "General"
if not res.get("description"):
    res["description"] = "No description"

print(res)

{'amount': 1000, 'category': 'General', 'date': '2024-12-22', 'description': 'No description'}


In [116]:
if res["category"] == "General" and res["description"] == "No description":
    final = llm_tools.invoke(
        f"{res} \nreview and refine this list according to initial input:\n{user_input}."
    ).tool_calls[0]["args"]
else:
    final = res

print(final)

{'amount': 1000, 'category': 'General', 'date': '2024-12-22', 'description': 'No description'}
